In [268]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.preprocessing import OneHotEncoder 
from sklearn import preprocessing

In [269]:
data = pd.read_csv('data_clean.csv')

In [270]:
data.head()

,Unnamed: 0,X,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,...,EnclosedPorch,X3SsnPorch,ScreenPorch,Fence,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,0,1,60,RL,65.0,8450,Pave,No Alley,Reg,...,0,0,0,No Fence,0,2,2008,WD,Normal,208500.0
1,2,1,2,20,RL,80.0,9600,Pave,No Alley,Reg,...,0,0,0,No Fence,0,5,2007,WD,Normal,181500.0
2,3,2,3,60,RL,68.0,11250,Pave,No Alley,IR1,...,0,0,0,No Fence,0,9,2008,WD,Normal,223500.0
3,4,3,4,70,RL,60.0,9550,Pave,No Alley,IR1,...,272,0,0,No Fence,0,2,2006,WD,Abnorml,140000.0
4,5,4,5,60,RL,84.0,14260,Pave,No Alley,IR1,...,0,0,0,No Fence,0,12,2008,WD,Normal,250000.0


In [360]:
train = data[0:1460]
test = data[1461:]
test = test.drop(["SalePrice"], axis = 1)

In [361]:
train_neigh1 = train[['Neighborhood', 'SalePrice']]
train_neigh1.head()

,Neighborhood,SalePrice
0,CollgCr,208500.0
1,Veenker,181500.0
2,CollgCr,223500.0
3,Crawfor,140000.0
4,NoRidge,250000.0


In [362]:
tn = train.Neighborhood.unique()
tn = pd.DataFrame(tn)
tn['mylablels']= list(range(1,26))
tn.columns = ['Neighborhood', 'mylabels']
train_neigh1 = train_neigh1.merge(tn, on = "Neighborhood")
train_neigh1 = train_neigh1[["mylabels", "SalePrice"]]

In [363]:
tn

,Neighborhood,mylabels
0,CollgCr,1
1,Veenker,2
2,Crawfor,3
3,NoRidge,4
4,Mitchel,5
5,Somerst,6
6,NWAmes,7
7,OldTown,8
8,BrkSide,9
9,Sawyer,10


In [365]:
train_neigh_means = train_neigh1.groupby("mylabels").agg("mean").reset_index().sort_values("SalePrice")
train_neigh_means

,mylabels,SalePrice
14,15,98576.470588
13,14,100123.783784
22,23,104493.750000
8,9,124834.051724
15,16,128219.700000
7,8,128225.300885
9,10,136793.135135
24,25,137500.000000
23,24,142591.360000
20,21,142694.444444


In [366]:
#LABEL ENCODING
#train_neigh = pd.get_dummies(train_neigh1['Neighborhood'])
#train_neigh['SalePrice'] = train_neigh1.SalePrice

In [367]:
train_neigh_means = train_neigh_means.to_numpy()

In [368]:
kmeans = KMeans(n_clusters=5, random_state=0).fit(train_neigh_means)
klabels = kmeans.labels_
klabels

array([3, 3, 3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 0, 0, 0, 0, 0, 0, 4, 4, 4,
       1, 1, 1], dtype=int32)

In [369]:
klabels = pd.DataFrame(klabels)

In [370]:
ktrain_neigh = pd.DataFrame(train_neigh_means)
ktrain_neigh['klabels'] = klabels

In [371]:
ktrain_neigh

,0,1,klabels
0,15.0,98576.470588,3
1,14.0,100123.783784,3
2,23.0,104493.750000,3
3,9.0,124834.051724,2
4,16.0,128219.700000,2
5,8.0,128225.300885,2
6,10.0,136793.135135,2
7,25.0,137500.000000,2
8,24.0,142591.360000,2
9,21.0,142694.444444,2


In [372]:
#DE-DUMMIFYING 
#ktrain_neigh['nbrhd'] = ""
#for col in ktrain_neigh.columns[:25]:
#    indices =  ktrain_neigh.loc[train_neigh[col]==1].index
#    for j in indices:
#        ktrain_neigh['nbrhd'][j] = col

In [373]:
#ktrain_neigh = ktrain_neigh[['nbrhd', 'klabels', 'SalePrice']]

In [374]:
ktrain_neigh.columns = ['mylabels', 'SalePrice', 'klabels']

In [375]:
ktrain_neigh.groupby("klabels").agg({"SalePrice":"mean", "mylabels":"nunique"})

,SalePrice,mylabels
klabels,,
0,197783.883026,7
1,320688.313483,3
2,138108.354960,9
3,101064.668124,3
4,235466.670617,3


In [376]:
train_neigh_dict = ktrain_neigh[['klabels', 'mylabels']]
train_neigh_dict = ktrain_neigh.drop_duplicates()
train_neigh_dict = train_neigh_dict.set_index('klabels').sort_values("SalePrice").reset_index()
train_neigh_dict = train_neigh_dict.merge(tn, on= 'mylabels')[['klabels','Neighborhood','SalePrice']]
train_neigh_dict

,klabels,Neighborhood,SalePrice
0,3,MeadowV,98576.470588
1,3,IDOTRR,100123.783784
2,3,BrDale,104493.750000
3,2,BrkSide,124834.051724
4,2,Edwards,128219.700000
5,2,OldTown,128225.300885
6,2,Sawyer,136793.135135
7,2,Blueste,137500.000000
8,2,SWISU,142591.360000
9,2,NPkVill,142694.444444
